<p style="font-family: Arial; font-size:3em;color:black;"> Lab Exercise 11</p>

#### Team Members

1. Kevin Xavier Antony Arul Xavier
2. Vikas Baliyan
3. Ramuel John Batuigas
4. Chirag Ashokkumar Bhatia
5. Jazbi Izhar
6. Sidhartha Sankar Mondal
7. Bharat Sharma

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(5)  # creating pseudo-random numbers for reproducibility

In [2]:
# Let's consider the following A_orig matrix:

A_orig = np.array([[4, 2, 4, 5, 4, 2, 2], [4, 4, 3, 2, 2, 5, 4], [4, 1, 4, 7, 3, 6, 2], [8 ,1, 2, 0, 5, 0, 7], [4 , 5, 8, 7, 6, 2, 3]],dtype=float)
M, N = 5, 7
print (pd.DataFrame(A_orig).head())

     0    1    2    3    4    5    6
0  4.0  2.0  4.0  5.0  4.0  2.0  2.0
1  4.0  4.0  3.0  2.0  2.0  5.0  4.0
2  4.0  1.0  4.0  7.0  3.0  6.0  2.0
3  8.0  1.0  2.0  0.0  5.0  0.0  7.0
4  4.0  5.0  8.0  7.0  6.0  2.0  3.0


In [3]:
# Let's add some NaN(s) to  A_orig matrix

A = A_orig.copy()
A[3][1] = np.NaN
A[4][6] = np.NaN

A_df = pd.DataFrame(A)
print (A_df.head())

     0    1    2    3    4    5    6
0  4.0  2.0  4.0  5.0  4.0  2.0  2.0
1  4.0  4.0  3.0  2.0  2.0  5.0  4.0
2  4.0  1.0  4.0  7.0  3.0  6.0  2.0
3  8.0  NaN  2.0  0.0  5.0  0.0  7.0
4  4.0  5.0  8.0  7.0  6.0  2.0  NaN


In [4]:
# Fine tune K value such that the (average percentage) error for the 2 missing elements is minimum.
# Remember: this is an exercise where we have the actual values for missing elements. 
# In real-life scenarios missins elements are not known and you may need different metric(s)
# to assess the quality of your reconstructed matrix.
K = 1
P = np.abs(np.random.uniform(low=0, high=8, size=(M, K)))
Q = np.abs(np.random.uniform(low=0, high=8, size=(K, N)))
P = np.divide(P, K*P.max())
Q = np.divide(Q, K*Q.max())

In [5]:
def matrix_factorization(Rating_Matrix, P, Q, K, steps, alpha=0.001, beta=0.02):
    Q = Q.T
    ErrorValues = []
    for step in range(steps):
        for i in range(len(Rating_Matrix)):
            for j in range(len(Rating_Matrix[i])):
                if ~np.isnan(Rating_Matrix[i][j]):
                    eij = Rating_Matrix[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eRating_Matrix = np.dot(P,Q)
        e = 0
        for i in range(len(Rating_Matrix)):
            for j in range(len(Rating_Matrix[i])):
                if ~np.isnan(Rating_Matrix[i][j]):
                    e = e + pow(Rating_Matrix[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        
        
        ErrorValues.append(e)
        
        print("Total error at step", step, "is", e)
        if e < 0.0001:
            break
    
    return P, Q.T

In [6]:
from sklearn.metrics import mean_absolute_percentage_error  

mapeData =  pd.DataFrame()
for K in range (1,15):
#     print(K)
    
    P = np.abs(np.random.uniform(low=0, high=8, size=(M, K)))
    Q = np.abs(np.random.uniform(low=0, high=8, size=(K, N)))
    P = np.divide(P, K*P.max())
    Q = np.divide(Q, K*Q.max())
    eP, eQ = matrix_factorization(A, P, Q.T, K, steps = 1000)
    eA = np.matmul(eP, eQ.T)
   
    # computing the errors
    y_original = [A_orig[3][1],A_orig[4][6]]
    y_predicted = [eA[3][1],eA[4][6]]
    
    print(y_original,y_predicted)
    mape = mean_absolute_percentage_error(y_original,y_predicted)
    mapeData = mapeData.append(pd.DataFrame({"K Value":K,"Mape Error":mape},index = [0]))
    
    
    
    print(A,'\n')
    print(eA)

Total error at step 0 is 553.6873926585794
Total error at step 1 is 546.0726940188043
Total error at step 2 is 538.0338844645186
Total error at step 3 is 529.558785994032
Total error at step 4 is 520.6382837142479
Total error at step 5 is 511.266809136036
Total error at step 6 is 501.4428347247918
Total error at step 7 is 491.1693651830846
Total error at step 8 is 480.454407948522
Total error at step 9 is 469.3114025837913
Total error at step 10 is 457.75958636782013
Total error at step 11 is 445.82427175687536
Total error at step 12 is 433.5370107839666
Total error at step 13 is 420.93562221215655
Total error at step 14 is 408.0640596220967
Total error at step 15 is 394.97210278710236
Total error at step 16 is 381.71486073873007
Total error at step 17 is 368.35208275794525
Total error at step 18 is 354.94728285518545
Total error at step 19 is 341.5666936368758
Total error at step 20 is 328.2780761162052
Total error at step 21 is 315.1494221849583
Total error at step 22 is 302.24759521

Total error at step 305 is 116.92073749755737
Total error at step 306 is 116.92071366111841
Total error at step 307 is 116.9206899974144
Total error at step 308 is 116.92066649934269
Total error at step 309 is 116.92064316012103
Total error at step 310 is 116.92061997327332
Total error at step 311 is 116.92059693261653
Total error at step 312 is 116.92057403224771
Total error at step 313 is 116.92055126653183
Total error at step 314 is 116.92052863009012
Total error at step 315 is 116.92050611778849
Total error at step 316 is 116.92048372472729
Total error at step 317 is 116.92046144623072
Total error at step 318 is 116.92043927783709
Total error at step 319 is 116.92041721528935
Total error at step 320 is 116.9203952545262
Total error at step 321 is 116.92037339167332
Total error at step 322 is 116.92035162303544
Total error at step 323 is 116.92032994508806
Total error at step 324 is 116.92030835447012
Total error at step 325 is 116.92028684797678
Total error at step 326 is 116.92026

Total error at step 665 is 116.91471434047655
Total error at step 666 is 116.91470214273522
Total error at step 667 is 116.9146899682382
Total error at step 668 is 116.91467781697094
Total error at step 669 is 116.91466568891872
Total error at step 670 is 116.91465358406695
Total error at step 671 is 116.9146415024011
Total error at step 672 is 116.91462944390652
Total error at step 673 is 116.91461740856847
Total error at step 674 is 116.91460539637248
Total error at step 675 is 116.91459340730385
Total error at step 676 is 116.91458144134796
Total error at step 677 is 116.9145694984902
Total error at step 678 is 116.91455757871596
Total error at step 679 is 116.91454568201057
Total error at step 680 is 116.9145338083594
Total error at step 681 is 116.91452195774784
Total error at step 682 is 116.9145101301613
Total error at step 683 is 116.91449832558507
Total error at step 684 is 116.91448654400459
Total error at step 685 is 116.91447478540525
Total error at step 686 is 116.91446304

Total error at step 0 is 605.8622954122094
Total error at step 1 is 602.7056362470877
Total error at step 2 is 599.3066827086441
Total error at step 3 is 595.6480290084776
Total error at step 4 is 591.7117217421757
Total error at step 5 is 587.4793482001561
Total error at step 6 is 582.9321509764213
Total error at step 7 is 578.0511721250823
Total error at step 8 is 572.8174300823827
Total error at step 9 is 567.2121323981291
Total error at step 10 is 561.2169269660956
Total error at step 11 is 554.8141938660191
Total error at step 12 is 547.9873790871154
Total error at step 13 is 540.7213702532857
Total error at step 14 is 533.0029129780455
Total error at step 15 is 524.8210646188392
Total error at step 16 is 516.1676799701538
Total error at step 17 is 507.0379208527901
Total error at step 18 is 497.4307786771057
Total error at step 19 is 487.34959597733473
Total error at step 20 is 476.80256977727475
Total error at step 21 is 465.80321665227126
Total error at step 22 is 454.370776747

Total error at step 275 is 39.88137815501246
Total error at step 276 is 39.86310050788895
Total error at step 277 is 39.84508741498347
Total error at step 278 is 39.82733059449269
Total error at step 279 is 39.809822131589016
Total error at step 280 is 39.792554459784064
Total error at step 281 is 39.77552034328087
Total error at step 282 is 39.75871286026341
Total error at step 283 is 39.74212538707601
Total error at step 284 is 39.725751583245426
Total error at step 285 is 39.70958537730222
Total error at step 286 is 39.69362095335884
Total error at step 287 is 39.67785273840475
Total error at step 288 is 39.66227539028045
Total error at step 289 is 39.646883786293756
Total error at step 290 is 39.63167301244424
Total error at step 291 is 39.61663835322301
Total error at step 292 is 39.60177528195657
Total error at step 293 is 39.58707945166538
Total error at step 294 is 39.57254668640892
Total error at step 295 is 39.55817297309109
Total error at step 296 is 39.54395445370024
Total 

...

Total error at step 79 is 127.06669167358424
Total error at step 80 is 125.83061545090682
Total error at step 81 is 124.74687831444055
Total error at step 82 is 123.79658321183724
Total error at step 83 is 122.96282546196814
Total error at step 84 is 122.23056609538895
Total error at step 85 is 121.58649253209208
Total error at step 86 is 121.01887322556657
Total error at step 87 is 120.51741150257226
Total error at step 88 is 120.07310257617326
Total error at step 89 is 119.67809662590915
Total error at step 90 is 119.32556992821732
Total error at step 91 is 119.00960527596524
Total error at step 92 is 118.72508233466952
Total error at step 93 is 118.46757812713422
Total error at step 94 is 118.23327749865847
Total error at step 95 is 118.01889317243872
Total error at step 96 is 117.82159484129555
Total error at step 97 is 117.63894664107147
Total error at step 98 is 117.46885229878013
Total error at step 99 is 117.30950723278451
Total error at step 100 is 117.1593568929853
Total erro

Total error at step 307 is 30.722027134895523
Total error at step 308 is 30.49954411841728
Total error at step 309 is 30.278264250492253
Total error at step 310 is 30.05809395147431
Total error at step 311 is 29.838946132301732
Total error at step 312 is 29.620740020507323
Total error at step 313 is 29.403400982089902
Total error at step 314 is 29.186860340183628
Total error at step 315 is 28.97105519137162
Total error at step 316 is 28.755928220408737
Total error at step 317 is 28.541427514040333
Total error at step 318 is 28.32750637452842
Total error at step 319 is 28.11412313342885
Total error at step 320 is 27.90124096609791
Total error at step 321 is 27.688827707347215
Total error at step 322 is 27.47685566861059
Total error at step 323 is 27.26530145693679
Total error at step 324 is 27.054145796074888
Total error at step 325 is 26.843373349877943
Total error at step 326 is 26.63297254821322
Total error at step 327 is 26.42293541553296
Total error at step 328 is 26.21325740222967

Total error at step 528 is 6.356376902898916
Total error at step 529 is 6.33354443140905
Total error at step 530 is 6.310964239051677
Total error at step 531 is 6.288634269272517
Total error at step 532 is 6.266552498449837
Total error at step 533 is 6.244716934000629
Total error at step 534 is 6.223125612564544
Total error at step 535 is 6.201776598264664
Total error at step 536 is 6.180667981042699
Total error at step 537 is 6.1597978750674764
Total error at step 538 is 6.1391644172146975
Total error at step 539 is 6.118765765616351
Total error at step 540 is 6.098600098277959
Total error at step 541 is 6.0786656117621485
Total error at step 542 is 6.058960519936575
Total error at step 543 is 6.039483052784575
Total error at step 544 is 6.020231455276909
Total error at step 545 is 6.00120398630273
Total error at step 546 is 5.982398917658207
Total error at step 547 is 5.963814533090994
Total error at step 548 is 5.945449127399036
Total error at step 549 is 5.927301005581872
Total err

Total error at step 752 is 4.735383509277387
Total error at step 753 is 4.734683435332334
Total error at step 754 is 4.73399488812281
Total error at step 755 is 4.733317650016614
Total error at step 756 is 4.732651507316597
Total error at step 757 is 4.731996250197266
Total error at step 758 is 4.731351672642063
Total error at step 759 is 4.730717572381498
Total error at step 760 is 4.730093750831981
Total error at step 761 is 4.729480013035507
Total error at step 762 is 4.728876167599926
Total error at step 763 is 4.728282026640192
Total error at step 764 is 4.727697405720193
Total error at step 765 is 4.727122123795405
Total error at step 766 is 4.726556003156314
Total error at step 767 is 4.725998869372534
Total error at step 768 is 4.725450551237696
Total error at step 769 is 4.724910880715105
Total error at step 770 is 4.724379692884093
Total error at step 771 is 4.723856825887088
Total error at step 772 is 4.723342120877467
Total error at step 773 is 4.7228354219681385
Total erro

Total error at step 969 is 4.683650942624291
Total error at step 970 is 4.683551355021721
Total error at step 971 is 4.683452017558488
Total error at step 972 is 4.683352927051699
Total error at step 973 is 4.6832540803770355
Total error at step 974 is 4.683155474467614
Total error at step 975 is 4.683057106312819
Total error at step 976 is 4.682958972957239
Total error at step 977 is 4.682861071499534
Total error at step 978 is 4.6827633990914155
Total error at step 979 is 4.68266595293659
Total error at step 980 is 4.682568730289701
Total error at step 981 is 4.682471728455404
Total error at step 982 is 4.682374944787297
Total error at step 983 is 4.682278376687044
Total error at step 984 is 4.6821820216033725
Total error at step 985 is 4.6820858770311835
Total error at step 986 is 4.681989940510636
Total error at step 987 is 4.681894209626269
Total error at step 988 is 4.6817986820061295
Total error at step 989 is 4.681703355320924
Total error at step 990 is 4.681608227283181
Total 

In [7]:
print(mapeData)

   K Value  Mape Error
0        1    1.386570
0        2    4.735211
0        3    4.574823
0        4    3.049942
0        5    2.695976
0        6    2.259773
0        7    2.742763
0        8    2.547217
0        9    2.776935
0       10    2.623045
0       11    2.857153
0       12    2.643901
0       13    2.640496
0       14    2.564710


## K after value of 1 the errors are minimised 